In [13]:
import numpy as np
import io
import scipy as sp
from scipy import sparse
from utils import kernel

def format_np_array(mat):
    s = ""
    for r in mat:
        s += "[" + ','.join([str(c) for c in r]) + "],\n"
    return s[:-2]

def format_np_array_compact(mat):
    s = ""
    for r in mat:
        s += ''.join([str(c) for c in r]) + "\n"
    return s[:-1]

def parse_magma_matrix(content, print_matrix=False):
    # Remove whitespace and newlines
    content = content.replace('\n', '').replace(' ', '').replace('][','\n')[1:-1]
    if print_matrix:
        print(content)
    mat = []
    for line in content.split('\n'):
        mat.append(np.array(list(line), dtype=int))
    mat = np.array(mat)
    print(mat.shape)
    return mat 

def read_in_mat(n):
    # from reader-friendly 0/1 txt to numpy array
    mat = []
    with open(f"{n}.txt") as file:
        for line in file:
            mat.append(np.array(list(line.rstrip('\n')), dtype=int))
    mat = np.array(mat)  
    print("read in matrix of shape", mat.shape)
    return mat
    
def save_mtx(mat, suffix='X'):
    fh = io.BytesIO()
    sp.io.mmwrite(fh, sparse.csr_matrix(mat))
    with open(f"{mat.shape[1]}_{suffix}.mtx", 'w') as f:
        print(fh.getvalue().decode('utf-8'), file=f, end="")
        
def save_triply_even(SX):
    # from the triply-even X-stabilizers, save both X and Z stabilizers in MTX format
    # the two MTX files will be used in verifying distance with QDistRnd
    n = SX.shape[1]
    print("X stabilizer shape", SX.shape)
    with open(f"{n}.txt", 'w') as f:
        print(format_np_array_compact(SX), file=f, end="")
    save_mtx(SX, suffix='X') # save X stabilizers in MTX format
    mat = np.vstack((np.ones((1,n),dtype=int), SX)) # stack with all-one vector (X-logical) 
    SZ = kernel(mat)[0] # Z stabilizers commute with (X stabilizers + X logical)
    print("Z stabilizer shape", SZ.shape)
    save_mtx(SZ, suffix='Z') # save Z stabilizers in MTX format
    print(f"saved to {n}_X.mtx and {n}_Z.mtx")
    
def doubling_transform(DE, TE):
    # input: X-stabilizers of doubly-even (DE) and triply-even (TE) codes
    # new triply-even codes with X-stabilizers:
    # DE DE 
    #       TE
    #    1  1
    m_DE, n_DE = DE.shape
    m_TE, n_TE = TE.shape
    row_1 = np.hstack((DE, DE, np.zeros((m_DE,n_TE), dtype=int)))
    row_2 = np.hstack((np.zeros((m_TE,n_DE),dtype=int), np.zeros((m_TE,n_DE),dtype=int), TE))
    row_3 = np.hstack((np.zeros((1,n_DE),dtype=int), np.ones((1,n_DE),dtype=int), np.ones((1,n_TE),dtype=int)))
    new_TE = np.vstack((row_1, row_2, row_3))
    save_triply_even(new_TE)
    return new_TE
    
    
    

In [14]:
SX = doubling_transform(read_in_mat(23), read_in_mat(49)) # get 95
SX = doubling_transform(read_in_mat(41), read_in_mat(95)) # get 177
SX = doubling_transform(read_in_mat(47), read_in_mat(177)) # get 271 
SX = doubling_transform(read_in_mat(65), read_in_mat(271)) # get 401
SX = doubling_transform(read_in_mat(79), read_in_mat(401)) # get 559

read in matrix of shape (11, 23)
read in matrix of shape (13, 49)
X stabilizer shape (25, 95)
Z stabilizer shape (69, 95)
saved to 95_X.mtx and 95_Z.mtx
read in matrix of shape (20, 41)
read in matrix of shape (25, 95)
X stabilizer shape (46, 177)
Z stabilizer shape (130, 177)
saved to 177_X.mtx and 177_Z.mtx
read in matrix of shape (23, 47)
read in matrix of shape (46, 177)
X stabilizer shape (70, 271)
Z stabilizer shape (200, 271)
saved to 271_X.mtx and 271_Z.mtx
read in matrix of shape (32, 65)
read in matrix of shape (70, 271)
X stabilizer shape (103, 401)
Z stabilizer shape (297, 401)
saved to 401_X.mtx and 401_Z.mtx
read in matrix of shape (39, 79)
read in matrix of shape (103, 401)
X stabilizer shape (143, 559)
Z stabilizer shape (415, 559)
saved to 559_X.mtx and 559_Z.mtx


In [83]:
# get generator matrix for extended QR codes from MAGMA online calculator
# C := QRCode(GF(2),23);
# MinimumWeight(C);
# GeneratorMatrix(C);
# copy the output
qr_mat = '''
[1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1]
[0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1 1]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1]
'''
qr_mat = parse_magma_matrix(qr_mat)
# then do shortening in python
for i in range(1,qr_mat.shape[0]):
    qr_mat[0] = (qr_mat[0]+qr_mat[i]) % 2
qr_mat = qr_mat[1:] # remove the all one row
qr_mat = qr_mat[:, 1:] # delete the first column
print(qr_mat.shape)
print(format_np_array_compact(qr_mat)) # copy the result to 23.txt

(12, 23)
(11, 22)
1000000000001100011101
0100000000011110110100
0010000000001111011010
0001000000000111101101
0000100000011011001100
0000010000001101100110
0000001000000110110011
0000000100011011100011
0000000010010101001011
0000000001010010011111
0000000000110001110101


In [15]:
mat = read_in_mat(66) # the matrix from "Extremal Self-Dual Codes of Lengths 66 and 68"
print(mat.sum(axis=1)) # print weight of each row (X stabilizer)
# print(format_np_array_compact(mat))
mat[-2] = (mat[-2] + mat[-1]) % 2
print(mat.sum(axis=1))
# print(format_np_array_compact(mat))
mat = mat[:-1] # drop the all-one row
all_zero_column = np.where(mat.sum(axis=0)==0)[0]
print(f"the column is all zero at {all_zero_column}")
mat = np.delete(mat, all_zero_column, axis=1)
print(format_np_array_compact(mat))
print(mat.sum(axis=1))
print(mat.shape)

save_mtx(mat, suffix='XZ') # save to 65_XZ.mtx

read in matrix of shape (33, 66)
[20 20 12 20 24 12 24 20 12 24 24 12 24 24 12 20 20 12 24 24 12 24 24 12
 20 24 12 24 20 12 20 38 66]
[20 20 12 20 24 12 24 20 12 24 24 12 24 24 12 20 20 12 24 24 12 24 24 12
 20 24 12 24 20 12 20 28 66]
the column is all zero at [31]
10000000000000000000000000000001000011101011101101011100001101011
01000000000000000000000000000001100010011110011011110010001011110
00100000000000000000000000000001010010100100100000100101001000100
00010000000000000000000000000001001010111001111101001110101001001
00001000000000000000000000000001100110110111010011111011011001111
00000100000000000000000000000001110000110000000100100001100001100
00000010000000000000000000000001011011110011101111001100111101101
00000001000000000000000000000001101110010010011010111010010011101
00000000100000000000000000000001110100100010100000000001000100101
00000000010000000000000000000001111001111010111101011100101111001
00000000001000000000000000000001111111010110110011110010011010111
000000